In [2]:
import django
import sys, os

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *

import pandas as pd
import numpy as np

In [4]:
columns = [
    'id',
    'UT_id',
    'authors',
    'content',
    'first_author',
    'title',
    'wosarticle__de',
    'wosarticle__wc'
]

In [13]:
q = Query.objects.get(pk=1457)

rated_docs = DocOwnership.objects.filter(tag__query=1457).values_list('doc__id',flat=True)

ratings = list(Doc.objects.filter(id__in=rated_docs,content__iregex='\w').values(*columns))
for r in ratings:
    d = Doc.objects.get(pk=r['id'])
    dos = DocOwnership.objects.filter(query=q,doc=d,relevant__gt=0)
    if dos.count() == 1:
        r['tag'] = dos.first().tag_id
        r['relevant'] = dos.first().relevant
        r['rated'] = dos.first().date
    elif dos.count() > 1:
        r['tag'] = dos.first().tag_id
        if dos.filter(relevant=1).count()>0:
            r['relevant'] = 1
            r['rated'] = dos.first().date
        else:
            r['relevant'] = dos.order_by('relevant').first().relevant
            r['rated'] = dos.first().date
    else:
        r['tag'] = ""
        r['relevant'] = 8
        r['rated'] = ""
        
df = pd.DataFrame.from_dict(ratings)

df = df[df['relevant'].isin([1,2])]

df['relevant'][df['relevant']==2] = 0

df = df.reset_index(drop=True)

df.head()


,UT_id,authors,content,first_author,id,rated,relevant,tag,title,wosarticle__de,wosarticle__wc
0,WOS:000239980800031,"Chan, APC, Chan, DWM, Fan, LCN, Lam, PTI, Yeun...",Project partnering has gained popularity in th...,"Chan, APC",88732,2018-09-21 02:32:31.206470+00:00,0,753,Partnering for construction excellence - A rea...,project partnering; partnering framework; comp...,[Construction & Building Technology; Engineeri...
1,WOS:000285823100009,"Kagan, S",Contemporary developments around the search fo...,"Kagan, S",229330,2018-09-21 12:42:54.004935+00:00,1,760,Cultures of sustainability and the aesthetics ...,None,[Economics; Planning & Development]
2,WOS:000270758400002,"Davis, JK, Green, JM",The effect caffeine elicits on endurance perfo...,"Davis, JK",629737,2018-09-21 13:10:17.460837+00:00,0,753,Caffeine and Anaerobic Performance Ergogenic V...,None,[Sport Sciences]
3,WOS:000330844100007,"Mace, PM, Sullivan, KJ, Cryer, M",New Zealand implemented a comprehensive manage...,"Mace, PM",231766,2018-09-21 03:07:17.635333+00:00,1,753,The evolution of New Zealands fisheries scienc...,fisheries management; ITQs; maximum sustainabl...,[Fisheries; Marine & Freshwater Biology; Ocean...
4,WOS:000350766100021,"Ley, SV, Fitzpatrick, DE, Ingham, RJ, Myers, RM",Organic synthesis is changing; in a world wher...,"Ley, SV",318012,2018-09-20 09:49:11.683386+00:00,1,760,Organic Synthesis: March of the Machines,analytical tools; automated optimization; cont...,"[Chemistry, Multidisciplinary]"


In [14]:
df.to_csv('docs.csv')